# `ContainsAll` Filter Demonstration

"The `ContainsAll` operator works on text properties and take an array of values as input. It will match objects where the property contains all of the values in the array." - [Weaviate Docs - April 18th, 2025](https://weaviate.io/developers/weaviate/search/filters#containsall-filter).

- This notebook illustrates how you can use this filter to search through a catalog of `Courses`

- This notebook additionally illustrates how you can combine Hybrid Search with the `ContainsAll` filter for more relevant search results while still matching **all** of the provided terms.

In [2]:
import os
from dotenv import load_dotenv
import weaviate

load_dotenv()

weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.environ.get("WEAVIATE_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.environ.get("WEAVIATE_API_KEY"))
)

In [8]:
import weaviate.classes as wvc
courses_collection = weaviate_client.collections.create(
    name="Courses",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_weaviate(),
    properties=[
        wvc.config.Property(
            name="courseTitle",
            data_type=wvc.config.DataType.TEXT,
        ),
        wvc.config.Property(
            name="courseDescription",
            data_type=wvc.config.DataType.TEXT,
        ),
        wvc.config.Property(
            name="courseDuration",
            data_type=wvc.config.DataType.NUMBER,
        ),
        wvc.config.Property(
            name="currentlyEnrolling",
            data_type=wvc.config.DataType.BOOL,
        ),
    ],
)

In [ ]:
# Toy Data
course_data = [
    {
        "courseTitle": "Introduction to Machine Learning",
        "courseDescription": "Learn the basics of machine learning algorithms and techniques. This course covers supervised and unsupervised learning approaches.",
        "courseDuration": 8,
        "currentlyEnrolling": True
    },
    {
        "courseTitle": "Advanced Deep Learning",
        "courseDescription": "Explore advanced neural network architectures and deep learning techniques. This course builds on fundamental machine learning concepts.",
        "courseDuration": 12,
        "currentlyEnrolling": True
    },
    {
        "courseTitle": "Natural Language Processing",
        "courseDescription": "Understand how to process and analyze text data using machine learning and deep learning approaches. This course covers text classification and generation.",
        "courseDuration": 10,
        "currentlyEnrolling": False
    },
    {
        "courseTitle": "Computer Vision Fundamentals",
        "courseDescription": "Learn image processing and computer vision techniques using deep learning. This course covers convolutional neural networks and object detection.",
        "courseDuration": 9,
        "currentlyEnrolling": True
    },
    {
        "courseTitle": "Reinforcement Learning",
        "courseDescription": "Master reinforcement learning algorithms and techniques. This course covers Q-learning and deep reinforcement learning approaches.",
        "courseDuration": 11,
        "currentlyEnrolling": False
    }
]

# Insert the data into the collection
for course in course_data:
    courses_collection.data.insert(properties=course)

### `ContainsAll` demonstration

In [10]:
# Example of using ContainsAll filter
from weaviate.classes.query import Filter

# Find courses where the description contains all of these terms
search_terms = ["deep", "learning"]

filtered_courses = courses_collection.query.fetch_objects(
    filters=Filter.by_property("courseDescription").contains_all(search_terms),
    limit=10
)

print("Courses containing all terms:", search_terms)
for course in filtered_courses.objects:
    print(f"- {course.properties['courseTitle']}: {course.properties['courseDescription']}")

# Another example with different terms
search_terms2 = ["machine", "learning", "techniques"]

filtered_courses2 = courses_collection.query.fetch_objects(
    filters=Filter.by_property("courseDescription").contains_all(search_terms2),
    limit=10
)

print("\nCourses containing all terms:", search_terms2)
for course in filtered_courses2.objects:
    print(f"- {course.properties['courseTitle']}: {course.properties['courseDescription']}")

Courses containing all terms: ['deep', 'learning']
- Advanced Deep Learning: Explore advanced neural network architectures and deep learning techniques. This course builds on fundamental machine learning concepts.
- Natural Language Processing: Understand how to process and analyze text data using machine learning and deep learning approaches. This course covers text classification and generation.
- Computer Vision Fundamentals: Learn image processing and computer vision techniques using deep learning. This course covers convolutional neural networks and object detection.
- Reinforcement Learning: Master reinforcement learning algorithms and techniques. This course covers Q-learning and deep reinforcement learning approaches.

Courses containing all terms: ['machine', 'learning', 'techniques']
- Introduction to Machine Learning: Learn the basics of machine learning algorithms and techniques. This course covers supervised and unsupervised learning approaches.
- Advanced Deep Learning: E

### `ContainsAll` with `str.split()` helper

The `ContainsAll` filter expects a text array as input. 

One way to convert a search query to a text array is with the `str.split()` method built into Python by default.

In [6]:
from weaviate.classes.query import Filter

# Find courses where the description contains all of these terms
search_query = "deep learning"
search_terms = search_query.split(" ")

filtered_courses = courses_collection.query.fetch_objects(
    filters=Filter.by_property("courseDescription").contains_all(search_terms),
    limit=10
)

print("Courses containing all terms:", search_terms)
for course in filtered_courses.objects:
    print(f"- {course.properties['courseTitle']}: {course.properties['courseDescription']}")


Courses containing all terms: ['deep', 'learning']
- Advanced Deep Learning: Explore advanced neural network architectures and deep learning techniques. This course builds on fundamental machine learning concepts.
- Natural Language Processing: Understand how to process and analyze text data using machine learning and deep learning approaches. This course covers text classification and generation.
- Computer Vision Fundamentals: Learn image processing and computer vision techniques using deep learning. This course covers convolutional neural networks and object detection.
- Reinforcement Learning: Master reinforcement learning algorithms and techniques. This course covers Q-learning and deep reinforcement learning approaches.


### Combine `ContainsAll` with Hybrid Search

Note how the ordering of search results are now ranked according to relevance to the query, **as well as** containing **all** of the provided terms.

In [13]:
# Example of using Hybrid Search with ContainsAll filter
from weaviate.classes.query import Filter

hybrid_query = "Reinforcement Learning"
search_terms = ["deep", "learning"]

hybrid_filtered_courses = courses_collection.query.hybrid(
    query=hybrid_query,
    filters=Filter.by_property("courseDescription").contains_all(search_terms),
    limit=10
)

print(f"\nHybrid search for '{hybrid_query}' containing all terms:", search_terms)
for course in hybrid_filtered_courses.objects:
    print(f"- {course.properties['courseTitle']}: {course.properties['courseDescription']}")

# Another example with different hybrid query and terms
hybrid_query2 = "Neural Network Architectures"
search_terms2 = ["machine", "learning", "techniques"]

hybrid_filtered_courses2 = courses_collection.query.hybrid(
    query=hybrid_query2,
    filters=Filter.by_property("courseDescription").contains_all(search_terms2),
    limit=10
)

print(f"\nHybrid search for '{hybrid_query2}' containing all terms:", search_terms2)
for course in hybrid_filtered_courses2.objects:
    print(f"- {course.properties['courseTitle']}: {course.properties['courseDescription']}")




Hybrid search for 'Reinforcement Learning' containing all terms: ['deep', 'learning']
- Reinforcement Learning: Master reinforcement learning algorithms and techniques. This course covers Q-learning and deep reinforcement learning approaches.
- Advanced Deep Learning: Explore advanced neural network architectures and deep learning techniques. This course builds on fundamental machine learning concepts.
- Computer Vision Fundamentals: Learn image processing and computer vision techniques using deep learning. This course covers convolutional neural networks and object detection.
- Natural Language Processing: Understand how to process and analyze text data using machine learning and deep learning approaches. This course covers text classification and generation.

Hybrid search for 'Neural Network Architectures' containing all terms: ['machine', 'learning', 'techniques']
- Advanced Deep Learning: Explore advanced neural network architectures and deep learning techniques. This course buil